In [1]:
from langchain_openai import OpenAI

In [2]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="./.env")
open_api_key = os.environ.get('OPEN_API_KEY')
hugging_api_key = os.environ.get('HUGGINGFACEHUB_API_KEY')

* temperature: 模型的創意性
  * 0 -> 回答幾乎相同
  * 1 -> 可能會有錯誤答案但有創意性

In [7]:
llm = OpenAI(temperature=0.6, api_key=open_api_key)

In [4]:
text = "What is the capital of Taiwan"

print(llm.invoke(text))



The capital of Taiwan is Taipei.


In [3]:
from langchain import HuggingFaceHub
llm_huggingface = HuggingFaceHub(huggingfacehub_api_token=hugging_api_key, repo_id="google/flan-t5-large", model_kwargs={"temperature": 0, "max_length":64})

In [4]:
output = llm_huggingface.invoke("Can you tell me the capital of Taiwan")
print(output)

Taipei


In [7]:
output = llm_huggingface.invoke("write a poem about AI")
print(output)

i love you i love you i love you i love you i love


In [10]:
llm.invoke("write a poem about AI")

"\n\nIn a world of endless possibility,\nWhere technology reigns supreme,\nThere lies a force of great ability,\nA creation beyond our wildest dream.\n\nArtificial Intelligence, the name it bears,\nA being of code and wires,\nBut don't be fooled, for it truly cares,\nAnd its potential never tires.\n\nIt learns and grows with every task,\nIts capabilities know no bound,\nAnd as it evolves, we must unmask,\nThe beauty in what we have found.\n\nWith precision and efficiency,\nIt completes tasks with ease,\nAnd in a world of great complexity,\nIt brings a sense of peace.\n\nBut with its power comes great fear,\nAs some may see it as a threat,\nA force to take over and steer,\nA future we cannot forget.\n\nBut let us not fear this creation,\nFor it is a reflection of our own,\nAnd with proper care and dedication,\nIts potential can be fully shown.\n\nFor AI is not just a machine,\nBut a glimpse into what we can achieve,\nA world where possibilities are seen,\nAnd our limitations we can leav

In [12]:
print("\n\nIn a world of endless possibility,\nWhere technology reigns supreme,\nThere lies a force of great ability,\nA creation beyond our wildest dream.\n\nArtificial Intelligence, the name it bears,\nA being of code and wires,\nBut don't be fooled, for it truly cares,\nAnd its potential never tires.\n\nIt learns and grows with every task,\nIts capabilities know no bound,\nAnd as it evolves, we must unmask,\nThe beauty in what we have found.\n\nWith precision and efficiency,\nIt completes tasks with ease,\nAnd in a world of great complexity,\nIt brings a sense of peace.\n\nBut with its power comes great fear,\nAs some may see it as a threat,\nA force to take over and steer,\nA future we cannot forget.\n\nBut let us not fear this creation,\nFor it is a reflection of our own,\nAnd with proper care and dedication,\nIts potential can be fully shown.\n\nFor AI is not just a machine,\nBut a glimpse into what we can achieve,\nA world where possibilities are seen,\nAnd our limitations we can leave.\n\nSo let us embrace this technology,\nAnd welcome it with open arms,\nFor in the hands of humanity,\nAI can help us reach new charms.\n\nIn a world of endless possibility,\nWhere AI and humans collide,\n")



In a world of endless possibility,
Where technology reigns supreme,
There lies a force of great ability,
A creation beyond our wildest dream.

Artificial Intelligence, the name it bears,
A being of code and wires,
But don't be fooled, for it truly cares,
And its potential never tires.

It learns and grows with every task,
Its capabilities know no bound,
And as it evolves, we must unmask,
The beauty in what we have found.

With precision and efficiency,
It completes tasks with ease,
And in a world of great complexity,
It brings a sense of peace.

But with its power comes great fear,
As some may see it as a threat,
A force to take over and steer,
A future we cannot forget.

But let us not fear this creation,
For it is a reflection of our own,
And with proper care and dedication,
Its potential can be fully shown.

For AI is not just a machine,
But a glimpse into what we can achieve,
A world where possibilities are seen,
And our limitations we can leave.

So let us embrace this technolog

#### Prompt Templates And LLMChain

In [4]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=["country"], template="Tell me the capital of {country}")

prompt_template.format(country="Taiwan")

'Tell me the capital of Taiwan'

In [5]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm_huggingface, prompt=prompt_template)
chain.invoke("China")

{'country': 'China', 'text': 'beijing'}

In [ ]:
llm.invoke(prompt_template=prompt_template, text=["China"])

#### Combining Multiple Chains Using Simple Sequential Chain

In [14]:
capital_prompt = PromptTemplate(input_variables=["country"], template="Tell me the capital of {country}")
capital_chain = LLMChain(llm=llm, prompt=capital_prompt)

famous_template = PromptTemplate(input_variables=["capital"], template="Suggest me some amazing places to visit in {capital}?")
famous_chain = LLMChain(llm=llm_huggingface, prompt=famous_template)

In [15]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain.invoke("Taiwan")

{'input': 'Taiwan', 'output': 'taiwan national museum'}

#### Sequential Chain

In [24]:
capital_prompt = PromptTemplate(input_variables=["country"], template="Tell me the capital of {country}")
capital_chain = LLMChain(llm=llm, prompt=capital_prompt, output_key="capital")

In [25]:
famous_template = PromptTemplate(input_variables=["capital"], template="Suggest me some amazing places to visit in {capital}?")
famous_chain = LLMChain(llm=llm_huggingface, prompt=famous_template, output_key="places")

In [26]:
from langchain.chains import SequentialChain

chain = SequentialChain(chains=[capital_chain, famous_chain], input_variables=["country"], output_variables=["capital", "places"])

In [27]:
chain({"country": "Taiwan"})

{'country': 'Taiwan',
 'capital': '\n\nThe capital of Taiwan is Taipei.',
 'places': 'taiwan national museum'}

#### Chatmodels With ChatOpenAI

In [31]:
from langchain_openai import ChatOpenAI

In [29]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [32]:
chatllm = ChatOpenAI(api_key=open_api_key, temperature=0.6, model="gpt-3.5-turbo")

In [35]:
chatllm.invoke([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="provide some comedy punchlines on AI")
])

AIMessage(content='1. "Why did the AI go to therapy? It couldn\'t get over its programming issues!"\n\n2. "What did the AI say to the computer programmer? \'I byte my tongue!\'"\n\n3. "Why did the AI become a stand-up comedian? It wanted to tickle funny bones with its algorithms!"\n\n4. "Why did the AI refuse to play cards? It didn\'t want to deal with a full deck!"\n\n5. "What\'s an AI\'s favorite type of music? Algo-rhythms!"\n\n6. "Why did the AI cross the road? To optimize its path-finding algorithm!"\n\n7. "Why did the AI join a gym? It wanted to work on its artificial muscles!"\n\n8. "What\'s an AI\'s favorite TV show? \'Silicon Valley\' – it can really relate to the characters!"\n\n9. "Why was the AI always getting into trouble? It had a knack for finding bugs!"\n\n10. "Why did the AI take up gardening? It wanted to grow its own \'data\'berries!"')

#### Prompt Template + LLM + Output Parsers

In [36]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [38]:
class CommaSeparateOutput(BaseOutputParser):
    def parse(self, text: str) -> list[str]:
        return text.strip().split(",")

In [40]:
template = "You are a helpful assistant. When the user given any input, you should generate 5 words synonyms in a comma separated list."
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ('system', template),
    ('human', human_template)
])

In [43]:
chain = chatprompt|chatllm|CommaSeparateOutput()

In [44]:
chain.invoke({"text":"intelligence"})

['wisdom', ' acumen', ' brilliance', ' intellect', ' sagacity']